In [1]:
import pandas as pd

In [4]:
d = {'City1': ['Paris/Lyon','Lyon/Paris'], 'City2': ['Lyon/Paris','Paris/Lyon'], 'bk': [1,2]}

In [10]:
dd =pd.DataFrame(d)

> can you create a sql to have City1 aggregate data which matches with the city2 values. So for dd dataframe, I expect to have one only city record and bk aggregated to that level. 

I'll help you create SQL-like aggregation using Python since we're working with a pandas DataFrame. We need to standardize the city pairs and then aggregate the bk values.

In [13]:
# First, let's standardize the city pairs by sorting them
dd['sorted_cities'] = dd.apply(lambda row: '/'.join(sorted(row['City1'].split('/'))), axis=1)

# Now aggregate the bk values
result = dd.groupby('sorted_cities')['bk'].sum().reset_index()
result.rename(columns={'sorted_cities': 'City_Pair'}, inplace=True)
result

,City_Pair,bk
0,Lyon/Paris,3


> can the create SQL for this. Python code not requried. 

Here's the SQL equivalent of the previous Python aggregation:

In [16]:
import sqlite3
from sqlite3 import Error
import pandas as pd

# Create a temporary SQLite database in memory
conn = sqlite3.connect(':memory:')

# Load the DataFrame into a SQL table
dd.to_sql('city_pairs', conn, index=False)

# Execute SQL query
sql_query = """
WITH normalized_pairs AS (
    CASE 
        WHEN City1 < City2 THEN City1 
        ELSE City2 
    END AS City_Pair,
    bk
FROM city_pairs
)
SELECT City_Pair, SUM(bk) as bk
FROM normalized_pairs
GROUP BY City_Pair;
"""

result_sql = pd.read_sql_query(sql_query, conn)
result_sql

DatabaseError: Execution failed on sql '
WITH normalized_pairs AS (
    CASE 
        WHEN City1 < City2 THEN City1 
        ELSE City2 
    END AS City_Pair,
    bk
FROM city_pairs
)
SELECT City_Pair, SUM(bk) as bk
FROM normalized_pairs
GROUP BY City_Pair;
': near "CASE": syntax error

In [19]:
import sqlite3
from sqlite3 import Error
import pandas as pd

# Create a temporary SQLite database in memory
conn = sqlite3.connect(':memory:')

# Load the DataFrame into a SQL table
dd.to_sql('city_pairs', conn, index=False)

# Execute SQL query
sql_query = """
WITH normalized_pairs AS (
    SELECT 
        CASE 
            WHEN City1 < City2 THEN City1 
            ELSE City2 
        END AS City_Pair,
        bk
    FROM city_pairs
)
SELECT City_Pair, SUM(bk) as bk
FROM normalized_pairs
GROUP BY City_Pair;
"""

result_sql = pd.read_sql_query(sql_query, conn)
result_sql

,City_Pair,bk
0,Lyon/Paris,3


Here's the SQL query that achieves the same result as the previous Python code. The query:
1. Creates a CTE (Common Table Expression) called normalized_pairs
2. Uses CASE statement to standardize the city pairs
3. Groups by the standardized city pair and sums the bk values

You can use this SQL query in any SQL environment that supports CTEs.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>